<a href="https://colab.research.google.com/github/Sam-Emma/WellaHealth/blob/main/fine_tuning_GPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install accelerate>=0.20.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.6 MB/s eta 0:00:00


In [2]:
!pip install -U PyPDF2
!pip install python-docx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 25.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=e8f58cefd9e4168c99d28284d8bed487172332be4ff28ec10c083716c54e2b5b
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [3]:
import pandas as pd
import numpy as np
import re
from PyPDF2 import PdfReader
import os
import docx

In [4]:
# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text


In [5]:
# Read documents from the directory
#train_directory = '/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/training_data/full_text'
train_directory = '/content/'
text_data = read_documents_from_directory(train_directory)
text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters


In [6]:
#text_data = read_pdf('/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/Cell_Biology.pdf')
#text_data = re.sub(r'\n+', '\n', text_data).strip()  # Remove excess newline characters

In [7]:
# Save the training and validation data as text files
#with open("/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/combined_text/full_text/train.txt", "w") as f:
 #   f.write(text_data)

In [8]:
with open("/content/data_syntax_prompt.txt", "w") as f:
    f.write(text_data)

In [9]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments

In [10]:
def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

In [11]:
def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=mlm,
    )
    return data_collator

In [12]:
def train(train_file_path,model_name,
          output_dir,
          overwrite_output_dir,
          per_device_train_batch_size,
          num_train_epochs,
          save_steps):
  tokenizer = GPT2Tokenizer.from_pretrained(model_name)
  train_dataset = load_dataset(train_file_path, tokenizer)
  data_collator = load_data_collator(tokenizer)

  tokenizer.save_pretrained(output_dir)

  model = GPT2LMHeadModel.from_pretrained(model_name)

  model.save_pretrained(output_dir)

  training_args = TrainingArguments(
          output_dir=output_dir,
          overwrite_output_dir=overwrite_output_dir,
          per_device_train_batch_size=per_device_train_batch_size,
          num_train_epochs=num_train_epochs,
      )

  trainer = Trainer(
          model=model,
          args=training_args,
          data_collator=data_collator,
          train_dataset=train_dataset,
  )

  trainer.train()
  trainer.save_model()

In [13]:
#train_file_path = "/content/drive/MyDrive/ColabNotebooks/data/chatbot_docs/combined_text/full_text/train.txt"
train_file_path = "/content/data_syntax_prompt.txt"
model_name = 'gpt2'
#output_dir = '/content/drive/MyDrive/ColabNotebooks/models/chat_models/custom_full_text'
output_dir = '/content/'
overwrite_output_dir = True
per_device_train_batch_size = 8
num_train_epochs = 100.0
save_steps = 50000

In [14]:
# Train
train(
    train_file_path=train_file_path,
    model_name=model_name,
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps
)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,0.630000


Step,Training Loss
500,0.630000
1000,0.072000
1500,0.033700


Inference

In [15]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel, GPT2TokenizerFast, GPT2Tokenizer

In [16]:
def load_model(model_path):
    model = GPT2LMHeadModel.from_pretrained(model_path)
    return model


def load_tokenizer(tokenizer_path):
    tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)
    return tokenizer

def generate_text(model_path, sequence, max_length):

    model = load_model(model_path)
    tokenizer = load_tokenizer(model_path)
    ids = tokenizer.encode(f'{sequence}', return_tensors='pt')
    final_outputs = model.generate(
        ids,
        do_sample=True,
        max_length=max_length,
        pad_token_id=model.config.eos_token_id,
        top_k=50,
        top_p=0.95,
    )
    #print(final_outputs)
    #print(tokenizer.decode(final_outputs[0], skip_special_tokens=True))
    output_ids = final_outputs[:, ids.shape[-1]:]  # Exclude input from the generated output
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    #print(output_text)
    return output_text

This model got trained on the entire text and took much longer to train, and yet it fails to give meaningful results.

In [17]:
# model1_path = "/content/drive/MyDrive/ColabNotebooks/models/chat_models/custom_full_text"
# sequence1 = "[Q] What is the Babel fish?"
# max_len = 50
# generate_text(model1_path, sequence1, max_len)

The following model was trained on 100 questions and answers based on the original text and it trained in a few seconds (50 epochs). It gives very meaningful results.

In [18]:
model2_path = "/content/"
sequence2 = """
User: how through what channel did they come?

"""
max_len = 200
result = generate_text(model2_path, sequence2, max_len)
print(result.split('[AI]:')[1].split('User:')[0][1:])

Through what channel did we come? Analyzing the results, we can calculate the total revenue generated by each channel. Here is a table showcasing the revenue distribution across different channels: [[channels&transactionRevenue?table]].



In [19]:
def get_response(question):
  model2_path = "/content/"
  sequence2 = f"""
  User: {question}

  """
  max_len = 200
  result = generate_text(model2_path, sequence2, max_len)
  return (result.split('[AI]:')[1].split('User:')[0][1:])


In [21]:
user_questions = [
    "What is the transaction revenue and number of orders for the current year? How many visits do we have in the last month?",
    "How do we increase transaction revenue this month?",
    "What were the pageviews and visitor count for the last year?",
    "Give me the transaction revenue and number of orders for the current month by country.",
    "Can you tell me the total sessions and unique users for the last year by browser?",
    "Provide me with the pageviews and visitor count for today by country."
]

for i in user_questions:
  print(f'User: {i}')
  print(f'AI: {get_response(i)}')
  print('\n')

User: What is the transaction revenue and number of orders for the current year? How many visits do we have in the last month?
AI: To determine the country with the highest revenue for the current year, we need to analyze the revenue by country. Based on the data, the country with the highest revenue for the current year is [[country&transactionRevenue]?top&sort=desc&period=current_year]."



User: How do we increase transaction revenue this month?
AI: Analyzing the performance of several crucial metrics, we identified that there was a [[transactionRevenue]?number&period=last&diff&percent] increase this month. This indicatesenne the revenue contribution of different payment methods. Here is a table summarizing the revenue: [[transactionRevenue]?top&sort&table].



User: What were the pageviews and visitor count for the last year?


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 10>:12                                                                            │
│ in get_response:9                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
IndexError: list index out of range